In [1]:
import tkinter as tk
from tkinter import messagebox
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

In [2]:
model_path = "./NLP_MAML_model"
maml_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
maml_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [4]:
def evaluate_sentiment():

    input_text = entry.get("1.0", "end-1c")

    if not input_text.strip():
        messagebox.showwarning("Input Error", "Please enter a sentence to evaluate!")
        return

    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt", max_length=512)

    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = maml_model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

    if predictions[0] == 1:
        result_label.config(text="Sentiment: Positive", fg="green")
    else:
        result_label.config(text="Sentiment: Negative", fg="red")

In [5]:
root = tk.Tk()
root.title("Sentiment Analysis")

root.geometry("500x500")

entry = tk.Text(root, height=15, width=50)
entry.pack(pady=20)

sentiment_button = tk.Button(root, text="Analyze Sentiment", command=evaluate_sentiment)
sentiment_button.pack(pady=10)

result_label = tk.Label(root, text="Sentiment: ", font=("Helvetica", 14))
result_label.pack(pady=10)

root.mainloop()